In [1]:
from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd

def get_client():
    fn = '../../key/binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


client = get_client()


def get_unix_timestamp(date_string):
    """
    Converts the input date string to Unix timestamp.

    Parameters:
        date_string (str): Input date string in the format "dd/mm/yyyy hh:mm:ss".

    Returns:
        int: Unix timestamp of the given date.
    """
    try:
        date_obj = datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S")
        timestamp = int(date_obj.timestamp())
        return timestamp
    except ValueError:
        print("Invalid date format. Please use the format 'dd/mm/yyyy hh:mm:ss'.")
        return None

def get_historical_data(start_timestamp, end_timestamp): 
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        klines = client.futures_historical_klines('BTCUSDT', '1h', current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    df.to_csv(f'../../data/1h.csv', index=False)
    print('Data Exported')



start_timestamp = get_unix_timestamp('1/10/2019 00:00:00')
end_timestamp = int(time())


get_historical_data(start_timestamp, end_timestamp)


ModuleNotFoundError: No module named 'binance'

In [3]:
import pandas as pd
import pandas_ta as ta


candlestick_frame = 24
pnl_threshold = 5


df = pd.read_csv('../../data/1h.csv')

df['ao'] = df.ta.ao()
df['apo'] = df.ta.apo()
df['bias'] = df.ta.bias()
df['bop'] = df.ta.bop()
df['cci'] = df.ta.cci()
df['cfo'] = df.ta.cfo()
df['cg'] = df.ta.cg()
df['cmo'] = df.ta.cmo()
df['coppock'] = df.ta.coppock()
df['cti'] = df.ta.cti()
df['er'] = df.ta.er()
df['inertia'] = df.ta.inertia()
macd = ta.macd(df['close'])
df['MACD_12_26_9'] = macd['MACD_12_26_9']
df['MACDh_12_26_9'] = macd['MACDh_12_26_9']
df['MACDs_12_26_9'] =macd['MACDs_12_26_9']
df['mom'] = df.ta.mom()
df['pgo'] = df.ta.pgo()
df['psl'] = df.ta.psl()
qqe = ta.qqe(df['close'])
df['QQE_14_5_4.236'] = qqe['QQE_14_5_4.236']
df['QQE_14_5_4.236_RSIMA'] = qqe['QQE_14_5_4.236_RSIMA']
df['QQEs_14_5_4.236'] = qqe['QQEs_14_5_4.236']
df['roc'] = df.ta.roc()
df['rsi'] = df.ta.rsi()
rvgi = ta.rvgi(high=df['high'], low=df['low'], close=df['close'], open_=df['open'])
df['RVGI_14_4'] = rvgi['RVGI_14_4']
df['RVGIs_14_4'] = rvgi['RVGIs_14_4'] 
df['slope'] = df.ta.slope()
smi = ta.smi(close=df['close'])
df['SMI_5_20_5'] = smi['SMI_5_20_5']
df['SMIs_5_20_5'] = smi['SMIs_5_20_5']
df['SMIo_5_20_5'] = smi['SMIo_5_20_5']
squeeze = ta.squeeze(high=df['high'], low=df['low'], close=df['close'])
df['SQZ_20_2.0_20_1.5'] = squeeze['SQZ_20_2.0_20_1.5']
df['SQZ_ON'] = squeeze['SQZ_ON']
df['SQZ_OFF'] = squeeze['SQZ_OFF']
df['SQZ_NO'] = squeeze['SQZ_NO']
squeeze_pro = ta.squeeze_pro(high=df['high'], low=df['low'], close=df['close'])
df['SQZPRO_20_2.0_20_2_1.5_1'] = squeeze_pro['SQZPRO_20_2.0_20_2_1.5_1']
df['SQZPRO_ON_WIDE'] = squeeze_pro['SQZPRO_ON_WIDE']
df['SQZPRO_ON_NORMAL'] = squeeze_pro['SQZPRO_ON_NORMAL']
df['SQZPRO_ON_NARROW'] = squeeze_pro['SQZPRO_ON_NARROW']
df['SQZPRO_OFF'] = squeeze_pro['SQZPRO_OFF']
df['SQZPRO_NO'] = squeeze_pro['SQZPRO_NO']
stc = ta.stc(close=df['close'])
df['STC_10_12_26_0.5'] = stc['STC_10_12_26_0.5']
df['STCmacd_10_12_26_0.5'] = stc['STCmacd_10_12_26_0.5']
df['STCstoch_10_12_26_0.5'] = stc['STCstoch_10_12_26_0.5']


ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
tot = len(ts)
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(high[idx:idx+candlestick_frame])
        min_low = min(low[idx:idx+candlestick_frame])
        entry_price = open[idx]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     


processed_df = df
long=[]
short=[]
dont_trade=[]
signal = []

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        if (long_runup_lst[idx] >= pnl_threshold) and (short_runup_lst[idx] >= pnl_threshold):
            signal.append('both')
        elif long_runup_lst[idx] >= pnl_threshold:
            signal.append('long')
        elif short_runup_lst[idx] >= pnl_threshold:
            signal.append('short')
        else:
            signal.append('dont_trade')
    else:
        signal.append('dont_trade')

df['signal'] = signal


df = df.drop(columns = ['time', 'open', 'high', 'low', 'close', 'volume'], axis=1)

print('saving dataset.csv....')
dataset_fn = f'../../data/dataset.csv'
pd.DataFrame(df).to_csv(dataset_fn, index=False)




saving dataset.csv....


In [ ]:
len(ts)==(len(long) + len(short) + len(dont_trade))

True

In [ ]:
(len(long) + len(short) + len(dont_trade))

34312